<a href="https://colab.research.google.com/github/RenanNB360/NLP_com_Bert/blob/main/Classifica%C3%A7%C3%A3o_de_Sentimentos_com_Bert_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Importações

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [2]:
!pip install bert-for-tf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=00d35dc0ab8efec9b0b92bff9083ed22562eea9979af36052d6745e2a620ef62
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=d1166bff65dbec8ea446f7d05423f8ab35b74577d37efe961994fe59c87da924
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=386e102ca19f3fb4b26f4089b0e0a2fa922bf8d282fb11647326fe1141c63e54
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e908768

In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
tf.__version__

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.14.0'

# Etapa 2: Pré-processamento

## Carregamento dos arquivos

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']
data = pd.read_csv('/content/drive/MyDrive/NLP/Bert/arquivo/training.1600000.processed.noemoticon.csv',
                   header = None,
                   names = cols,
                   engine = 'python',
                   encoding = 'latin1')

In [8]:
data.drop(['id', 'date', 'query', 'user'], axis = 1, inplace = True)

In [9]:
data.shape

(1600000, 2)

In [10]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Pré-processamento

### Limpeza

In [11]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text()
  tweet = re.sub(r'@[A-Za-z0-9]+', ' ', tweet)
  tweet = re.sub(r'https?://[A-Za-z0-9./]', ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r' +', ' ', tweet)
  return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

<ipython-input-11-d7c6d529435d>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, 'lxml').get_text()


In [13]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenização

In [15]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1',
                            trainable = False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [16]:
def encode_sentence(sent):
  return ['[CLS]'] + tokenizer.tokenize(sent) + ['[SEP]']

In [17]:
encode_sentence('My dog likes strawberries.')

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [18]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [19]:
print(data_inputs[0:2])

[['[CLS]', 'wit', '##pic', '.', 'com', 'y', 'z', '##l', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


### Criação da base de dados

In [23]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [26]:
get_ids(tokenizer.tokenize("My dog likes strawberries."))

[2026, 3899, 7777, 13137, 20968, 1012]

In [27]:
np.char.not_equal("[PAD]", "[PAD]")

array(False)

In [24]:
def get_mask(tokens):
  return np.char.not_equal(tokens, '[PAD]').astype(int)

In [28]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))

array([1, 1, 1, 1, 1, 1])

In [25]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == '[SEP]':
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [29]:
print(data_inputs[0])

['[CLS]', 'wit', '##pic', '.', 'com', 'y', 'z', '##l', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [30]:
get_segments(data_inputs[0])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [31]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red.") + ["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']

In [32]:
bert_layer([
          tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
          tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
          tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0),
          ])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935600e-01, -4.10335362e-01, -9.65754986e-01,
          9.07317877e-01,  8.12913775e-01, -1.74174353e-01,
          9.11234498e-01,  3.41952175e-01, -8.74521136e-01,
         -9.99989271e-01, -7.78409779e-01,  9.69385147e-01,
          9.86160576e-01,  6.36962950e-01,  9.48631287e-01,
         -7.51193225e-01, -4.58339483e-01, -7.08104610e-01,
          4.62098330e-01, -6.57927394e-01,  7.60414660e-01,
          9.99994814e-01, -3.96860749e-01,  3.44166070e-01,
          6.16488695e-01,  9.94400144e-01, -7.76633799e-01,
          9.38316584e-01,  9.59452271e-01,  7.32879341e-01,
         -6.93436742e-01,  2.93080479e-01, -9.93785441e-01,
         -1.64551854e-01, -9.67019439e-01, -9.95549619e-01,
          5.32935441e-01, -6.88061118e-01,  1.34714833e-02,
          2.98193898e-02, -9.18356538e-01,  4.20526177e-01,
          9.99988973e-01,  2.52676398e-01,  6.06235325e-01,
         -3.50750268e-01, -1.00000000e+00,  4.975

In [34]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],
              sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [35]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, output_types = (tf.int32, tf.int32))

In [38]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes = ((3, None), ()),
                                     padding_values = (0, 0))

In [39]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Etapa 3: Construção do modelo

In [50]:
class DCNNBERTEmbedding(tf.keras.Model):

  def __init__(self,
               nb_filters = 50,
               FFN_units = 512,
               nb_classes = 2,
               dropout_rate = 0.1,
               name = 'dcnn'):
    super(DCNNBERTEmbedding, self).__init__(name=name)

    self.bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1',
                                     trainable = False)

    self.bigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 2,
                                padding = 'valid',
                                activation = 'relu')
    self.trigram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 3,
                                padding = 'valid',
                                activation = 'relu')
    self.fourgram = layers.Conv1D(filters = nb_filters,
                                kernel_size = 4,
                                padding = 'valid',
                                activation = 'relu')
    self.pool = layers.GlobalMaxPool1D()
    self.dense_1 = layers.Dense(units = FFN_units, activation = 'relu')
    self.dropout = layers.Dropout(rate = dropout_rate)
    if nb_classes == 2:
      self.last_dense = layers.Dense(units = 1, activation = 'sigmoid')
    else:
      self.last_dense = layers.Dense(units = nb_classes, activation = 'softmax')

  def embed_with_bert(self, all_tokens):
    _, embs = self.bert_layer([all_tokens[:, 0, :],
                                all_tokens[:, 1, :],
                                all_tokens[:, 2, :]])

    return embs

  def call(self, inputs, training):
    x = self.embed_with_bert(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis = -1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training)
    output = self.last_dense(merged)

    return output

# Etapa 4: Treinamento

In [51]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [52]:
Dcnn = DCNNBERTEmbedding(nb_filters = NB_FILTERS,
                         FFN_units = FFN_UNITS,
                         nb_classes = NB_CLASSES,
                         dropout_rate = DROPOUT_RATE)

In [53]:
if NB_CLASSES == 2:
  Dcnn.compile(loss =  'binary_crossentropy', optimizer =  'adam', metrics = ['accuracy'])
else:
  Dcnn.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['sparse_categorical_accuracy'])

In [54]:
checkpoint_path = "/content/drive/MyDrive/NLP/Bert/arquivo/Embedding"

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!')

In [55]:
class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs = None):
    ckpt_manager.save()
    print(f'Checkpoint saved at {checkpoint_path}.')

In [57]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   steps_per_epoch = 500,
                   callbacks=[MyCustomCallback()])

Epoch 1/5
500/500 [==============================] - 48s 45ms/step - loss: 0.4544 - accuracy: 0.7934
Epoch 2/5
500/500 [==============================] - 24s 47ms/step - loss: 0.3906 - accuracy: 0.8287
Epoch 3/5
500/500 [==============================] - 25s 50ms/step - loss: 0.3943 - accuracy: 0.8254
Epoch 4/5
500/500 [==============================] - 20s 40ms/step - loss: 0.3914 - accuracy: 0.8292
Epoch 5/5
500/500 [==============================] - 20s 40ms/step - loss: 0.3777 - accuracy: 0.8359


# Etapa 5: Avaliação do modelo

In [58]:
results = Dcnn.evaluate(test_dataset)
print(results)

4537/4537 [==============================] - 120s 26ms/step - loss: 0.3607 - accuracy: 0.8428
[0.3606535494327545, 0.8427512645721436]


In [59]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))

In [60]:
get_prediction("This actor is very bad.")

Output of the model: [[0.04972439]]
Predicted sentiment: negative


In [61]:
get_prediction("I like dogs.")

Output of the model: [[0.9490403]]
Predicted sentiment: positive
